In [272]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [273]:
data = pd.read_csv('Walmart_Store_sales.csv')

In [274]:
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


### Part 1 : EDA and data preprocessing

Start your project by exploring your dataset : create figures, compute some statistics etc...

Then, you'll have to make some preprocessing on the dataset. You can follow the guidelines from the *preprocessing template*. There will also be some specific transformations to be planned on this dataset, for example on the *Date* column that can't be included as it is in the model. Below are some hints that might help you 🤓



    Store - the store number
    Date - the week of sales
    Weekly_Sales - sales for the given store
    Holiday_Flag - whether the week is a special holiday week 1 – Holiday week 0 – Non-holiday week
    Temperature - Temperature on the day of sale
    Fuel_Price - Cost of fuel in the region
    CPI – Prevailing consumer price index
    Unemployment - Prevailing unemployment rate
    Holiday Events\
    Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13\
    Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13\
    Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13\
    Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13


In [275]:
print(f"Shape of data set : {data.shape}")
print(data.columns)
display(data.describe(include='all'))
display(data.head())
data.isna().sum()

Shape of data set : (150, 8)
Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

#### Preprocessing to be planned with pandas

 **Drop lines where target values are missing :**
 - Here, the target variable (Y) corresponds to the column *Weekly_Sales*. One can see above that there are some missing values in this column.
 - We never use imputation techniques on the target : it might create some bias in the predictions !
 - Then, we will just drop the lines in the dataset for which the value in *Weekly_Sales* is missing.
 

In [276]:
# 1. drop target values that are missing
data = data.dropna(subset=['Weekly_Sales'])
data.isna().sum()

Store            0
Date            18
Weekly_Sales     0
Holiday_Flag    11
Temperature     15
Fuel_Price      12
CPI             11
Unemployment    14
dtype: int64

In [277]:
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896


In [278]:
# changement d'unité Température F en degrés Celsius
data['Temperature'] = 5/9 * (data['Temperature']- 32)

In [279]:
data['Temperature'].describe()

count    121.000000
mean      16.029982
std       10.285795
min       -7.338889
25%        7.344444
50%       16.805556
75%       24.416667
max       33.138889
Name: Temperature, dtype: float64

**Drop lines containing invalid values or outliers :**
In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. This concerns the columns : *Temperature*, *Fuel_price*, *CPI* and *Unemployment*
 


**Target variable/target (Y) that we will try to predict, to separate from the others** : *Weekly_Sales*

 **------------**


In [280]:
# Visualize pairwise dependencies before taking out outliers
fig = px.scatter_matrix(data)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

In [281]:
col = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
bornes = pd.DataFrame()
bornes['Feature'] = col
for i,c in enumerate(col) : 
    m = data[c].mean()
    s = data[c].std()
    bornes.loc[i,'min'] = m - 3 * s
    bornes.loc[i,'max'] = m + 3 * s
bornes


,Feature,min,max
0,Temperature,-14.827405,46.887368
1,Fuel_Price,1.878371,4.755613
2,CPI,57.361830,298.820458
3,Unemployment,2.807297,12.523867


In [282]:
for i,c in enumerate(col) : 
    data = data[(data[c] >= bornes.loc[i, 'min']) & (data[c] <= bornes.loc[i, 'max'])]

In [283]:
data.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,90.000000,9.000000e+01,80.000000,90.000000,90.000000,90.000000,90.000000
mean,9.900000,1.233865e+06,0.075000,16.145000,3.318444,179.524905,7.389733
std,6.204475,6.647250e+05,0.265053,9.858911,0.484399,39.554303,0.982729
min,1.000000,2.689290e+05,0.000000,-7.338889,2.548000,126.128355,5.143000
25%,4.000000,5.617240e+05,0.000000,7.412500,2.814750,132.602339,6.642250
50%,9.000000,1.260826e+06,0.000000,16.361111,3.468000,197.166416,7.419000
75%,15.750000,1.807159e+06,0.000000,24.329167,3.737750,214.855374,8.099000
max,20.000000,2.771397e+06,1.000000,33.138889,4.170000,226.968844,9.342000


In [284]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(data)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

On a bien réussi a enlever les outliers, selon leur définition.

In [285]:
# Correlation matrix
corr_matrix = data.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

**Create usable features from the *Date* column :**
The *Date* column cannot be included as it is in the model. Either you can drop this column, or you will create new columns that contain the following numeric features : 
- *year*
- *month*
- *day*
- *day of week*

In [286]:
# Transform Date column into year/month/day/day of week
data['Date'] = pd.to_datetime(data['Date'])
# Create new columns
data['day'] = data['Date'].dt.day
data['month'] = data['Date'].dt.month
data['year'] = data['Date'].dt.year
data['weekday'] = data['Date'].dt.weekday

C:\Users\Celine\AppData\Local\Temp\ipykernel_13952\1860900679.py:2: UserWarning:

Parsing '18-02-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

C:\Users\Celine\AppData\Local\Temp\ipykernel_13952\1860900679.py:2: UserWarning:

Parsing '25-03-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

C:\Users\Celine\AppData\Local\Temp\ipykernel_13952\1860900679.py:2: UserWarning:

Parsing '28-05-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

C:\Users\Celine\AppData\Local\Temp\ipykernel_13952\1860900679.py:2: UserWarning:

Parsing '19-08-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

C:\Users\Celine\AppData\Local\Temp\ipykernel_13952\1860900679.py:2: UserWarning:

Parsing '15-10-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consis

In [287]:
data = data.drop(['Date'], axis=1)

In [288]:
# Correlation matrix
corr_matrix = data.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

In [289]:
data.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day,month,year,weekday
0,6.0,1572117.54,NaN,15.338889,3.045,214.777523,6.858,18.0,2.0,2011.0,4.0
1,13.0,1807545.43,0.0,5.766667,3.435,128.616064,7.470,25.0,3.0,2011.0,4.0
4,6.0,1644470.66,0.0,26.050000,2.759,212.412888,7.092,28.0,5.0,2010.0,4.0
6,15.0,695396.19,0.0,21.000000,4.069,134.855161,7.658,6.0,3.0,2011.0,6.0
7,20.0,2203523.20,0.0,4.405556,3.617,213.023622,6.961,2.0,3.0,2012.0,4.0


In [290]:
data.shape

(90, 11)

#### Preprocessings to be planned with scikit-learn

 **Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

 - Categorical variables : Store, Holiday_Flag
 - Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek

In [291]:
data.columns

Index(['Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'day', 'month', 'year', 'weekday'],
      dtype='object')

In [292]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = data.drop(target_variable, axis = 1)
Y = data.loc[:,target_variable]

print("...Done.")
print()

# First : always divide dataset into train set & test set !!
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

print("...Done.")
print()


# Create pipeline for numeric features
numeric_features = ['Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'day', 'month', 'year', 'weekday'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


# Create pipeline for categorical features
categorical_features = ['Store','Holiday_Flag'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])


# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("Preprocessing done")

Separating labels from features...
...Done.

Dividing into train and test sets...
...Done.

Preprocessing done


In [293]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
127   16.0           0.0    16.550000       2.711  189.523128         6.868   
63     5.0           0.0    20.650000       3.594  224.019287         5.422   
35    19.0           0.0     0.700000       3.789  133.958742         7.771   
10     8.0           0.0    28.288889       3.554  219.070197         6.425   
95     1.0           0.0    23.766667       2.854  210.337426         7.808   

      day  month    year  weekday  
127   7.0    9.0  2010.0      1.0  
63   19.0   10.0  2012.0      4.0  
35   25.0    3.0  2011.0      4.0  
10   19.0    8.0  2011.0      4.0  
95   14.0    5.0  2010.0      4.0  
...Done.
[[ 0.04260362 -1.26840641  0.20507788 -0.55534542 -1.10466577  0.74406169
  -1.1763434  -2.03634567  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          1.          0

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



### Part 2 : Baseline model (linear regression)
Once you've trained a first model, don't forget to assess its performances on the train and test sets. Are you satisfied with the results ?
Besides, it would be interesting to analyze the values of the model's coefficients to know what features are important for the prediction. To do so, the `.coef_` attribute of scikit-learn's LinearRegression class might be useful. Please refer to the following link for more information 😉 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html



In [294]:
# Train model
print("Train model...")
regressor = LinearRegression() #permet de déclarer une instance de cette classe
regressor.fit(X_train, Y_train)
print("...Done.")

scores = cross_val_score(regressor, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

# Print R^2 scores
print("R2 score on training set : ", regressor.score(X_train, Y_train))
print("R2 score on test set : ", regressor.score(X_test, Y_test))

Train model...
...Done.
The cross-validated R2-score is :  0.9560063841841719
The standard deviation is :  0.03335914752216178
R2 score on training set :  0.9872729747897621
R2 score on test set :  0.9191907752940761


In [295]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'day', 'month', 'year', 'weekday']
Found categorical features  []


In [296]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num' : # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
        print(pipeline.named_steps['encoder'].get_feature_names_out())
    column_names.extend(features) # concatenate features names
        
#print("Names of columns corresponding to each coefficient: ", column_names)

['x0_2.0' 'x0_3.0' 'x0_4.0' 'x0_5.0' 'x0_6.0' 'x0_7.0' 'x0_8.0' 'x0_9.0'
 'x0_10.0' 'x0_11.0' 'x0_13.0' 'x0_14.0' 'x0_15.0' 'x0_16.0' 'x0_17.0'
 'x0_18.0' 'x0_19.0' 'x0_20.0' 'x1_1.0']


In [297]:
coef = pd.DataFrame(regressor.coef_)
coef.index = column_names
coef.columns = ['LinearReg']
coef
result = pd.DataFrame(columns=  ['model', 'accuracy', 'set'])
result.loc[len(result), result.columns] = 'LinearReg',  regressor.score(X_train, Y_train), 'train'
result.loc[len(result), result.columns] = 'LinearReg',  regressor.score(X_test, Y_test), 'test'
result

,model,accuracy,set
0,LinearReg,0.987273,train
1,LinearReg,0.919191,test


Selon ces coefficients, on voit que ce qui joue le plus sur la target sera le numéro du store. Puis ensuite, le CPI vient en premier ! 
Puis le jour de la semaine, le fuel price, l'unemployemnt, et en tout dernier, la température.

### Part 3 : Fight overfitting
In this last part, you'll have to train a **regularized linear regression model**. You'll find below some useful classes in scikit-learn's documentation :
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso



1/ Regularized linear regression : Ridge

In [298]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor_r = Ridge()
scores = cross_val_score(regressor_r, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

# Perform grid search
print("Grid search...")
regressor_r = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch_r = GridSearchCV(regressor_r, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch_r.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_r.best_params_)
print("Best R2 score : ", gridsearch_r.best_score_)



3-fold cross-validation...
The cross-validated R2-score is :  0.8407520375791133
The standard deviation is :  0.07538261768181109
Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.8262927909736191


In [299]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor_rg = Ridge(alpha=0.1)
scores = cross_val_score(regressor_rg, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.9533141353785968
The standard deviation is :  0.03206083040744807


In [300]:
# Print R^2 scores
print("R2 score on training set : ", gridsearch_r.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch_r.score(X_test, Y_test))


R2 score on training set :  0.983418228704619
R2 score on test set :  0.9185639095967127


In [301]:
gridsearch_r.best_estimator_.coef_

array([   10375.90700907,   -47085.11370895,    55581.98125082,
          73239.5980669 ,   -45195.54075363,    16873.36283105,
          46908.75632966,    -8235.53402555,   288436.35500069,
       -1059479.68304214,   837298.30127227, -1060151.42129084,
         183037.73347254,  -935722.12206249,  -485218.48536334,
        -905020.30601499,   292684.29813018,   339115.93086897,
         696863.26573839,   515936.16171316,  -712922.25835385,
        -816511.60826682,  -434478.79518963,  -369386.50463849,
          28144.93025564,   580128.45572424,   -80200.01384352])

In [302]:
coef['Ridge'] = gridsearch_r.best_estimator_.coef_
print(coef)

                 LinearReg         Ridge
Temperature  -1.546522e+03  1.037591e+04
Fuel_Price   -4.955352e+04 -4.708511e+04
CPI           8.955058e+05  5.558198e+04
Unemployment  3.444902e+04  7.323960e+04
day          -5.022475e+04 -4.519554e+04
month         6.616676e+03  1.687336e+04
year         -3.417536e+04  4.690876e+04
weekday      -1.989193e+04 -8.235534e+03
x0_2.0        2.707251e+05  2.884364e+05
x0_3.0       -1.269752e+06 -1.059480e+06
x0_4.0        2.604659e+06  8.372983e+05
x0_5.0       -1.227230e+06 -1.060151e+06
x0_6.0        8.974071e+04  1.830377e+05
x0_7.0       -5.046931e+05 -9.357221e+05
x0_8.0       -6.770598e+05 -4.852185e+05
x0_9.0       -1.133818e+06 -9.050203e+05
x0_10.0       2.211179e+06  2.926843e+05
x0_11.0       2.753377e+05  3.391159e+05
x0_13.0       2.492067e+06  6.968633e+05
x0_14.0       1.152157e+06  5.159362e+05
x0_15.0       9.726105e+05 -7.129223e+05
x0_16.0      -5.001510e+05 -8.165116e+05
x0_17.0       1.280854e+06 -4.344788e+05
x0_18.0       1.

In [303]:
result.loc[len(result), result.columns] = 'Ridge',  gridsearch_r.score(X_train, Y_train), 'train'
result.loc[len(result), result.columns] = 'Ridge',  gridsearch_r.score(X_test, Y_test), 'test'
result

,model,accuracy,set
0,LinearReg,0.987273,train
1,LinearReg,0.919191,test
2,Ridge,0.983418,train
3,Ridge,0.918564,test


2/ Regularized linear regression : Lasso

In [304]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor_l = Lasso()
scores = cross_val_score(regressor_l, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.9597392391812992
The standard deviation is :  0.02387758313205184


c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.848e+11, tolerance: 2.754e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.574e+11, tolerance: 2.737e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.552e+11, tolerance: 2.680e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model

In [305]:
# Perform grid search
print("Grid search...")
regressor_l = Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.1, 0.5, 0.7, 1] # 0 corresponds to no regularization
}
gridsearch_l = GridSearchCV(regressor_l, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch_l.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_l.best_params_)
print("Best R2 score : ", gridsearch_l.best_score_)


Grid search...


c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.048e+11, tolerance: 2.132e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.164e+11, tolerance: 1.941e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.014e+10, tolerance: 1.907e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model

...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.9177237506703664


c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.049e+11, tolerance: 2.132e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.162e+11, tolerance: 1.941e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.975e+10, tolerance: 1.907e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model

In [306]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor_lg = Lasso(alpha=0.1)
scores = cross_val_score(regressor_lg, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

# Print R^2 scores
print("R2 score on training set : ", gridsearch_l.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch_l.score(X_test, Y_test))


3-fold cross-validation...


c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.849e+11, tolerance: 2.754e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.568e+11, tolerance: 2.737e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.551e+11, tolerance: 2.680e+09

c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model

The cross-validated R2-score is :  0.959736484360023
The standard deviation is :  0.023876203626578028
R2 score on training set :  0.9867504854165919
R2 score on test set :  0.9233595891939552


c:\Users\Celine\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.242e+11, tolerance: 2.709e+09



In [307]:
coef['Lasso'] = gridsearch_l.best_estimator_.coef_
print(coef)
result.loc[len(result), result.columns] = 'Lasso',  gridsearch_l.score(X_train, Y_train), 'train'
result.loc[len(result), result.columns] = 'Lasso',  gridsearch_l.score(X_test, Y_test), 'test'
result

                 LinearReg         Ridge         Lasso
Temperature  -1.546522e+03  1.037591e+04 -1.109312e+03
Fuel_Price   -4.955352e+04 -4.708511e+04 -4.232341e+04
CPI           8.955058e+05  5.558198e+04  4.483216e+05
Unemployment  3.444902e+04  7.323960e+04  2.745478e+04
day          -5.022475e+04 -4.519554e+04 -4.973614e+04
month         6.616676e+03  1.687336e+04  1.205933e+04
year         -3.417536e+04  4.690876e+04 -7.714166e+03
weekday      -1.989193e+04 -8.235534e+03 -1.805619e+04
x0_2.0        2.707251e+05  2.884364e+05  2.516396e+05
x0_3.0       -1.269752e+06 -1.059480e+06 -1.223018e+06
x0_4.0        2.604659e+06  8.372983e+05  1.590363e+06
x0_5.0       -1.227230e+06 -1.060151e+06 -1.226272e+06
x0_6.0        8.974071e+04  1.830377e+05  8.273317e+04
x0_7.0       -5.046931e+05 -9.357221e+05 -7.490945e+05
x0_8.0       -6.770598e+05 -4.852185e+05 -6.598735e+05
x0_9.0       -1.133818e+06 -9.050203e+05 -1.115089e+06
x0_10.0       2.211179e+06  2.926843e+05  1.188990e+06
x0_11.0   

,model,accuracy,set
0,LinearReg,0.987273,train
1,LinearReg,0.919191,test
2,Ridge,0.983418,train
3,Ridge,0.918564,test
4,Lasso,0.98675,train
5,Lasso,0.92336,test


**Bonus question**

In regularized regression models, there's a hyperparameter called *the regularization strength* that can be fine-tuned to get the best generalized predictions on a given dataset. This fine-tuning can be done thanks to scikit-learn's GridSearchCV class : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Also, you'll find here some examples of how to use GridSearchCV together with Ridge or Lasso models : https://alfurka.github.io/2018-11-18-grid-search/

# Conclusion

* avec modèle linéaire classique : 
    * std = 0.033
    * R² (train) = 0.987
    * R² (test) = 0.919
    * 27 coef (27 variables)
* avec ridge 
    * alpha = 1 par défaut : R² = 0.84 (std = 0.075)
    * alpha = 0.1 : R² = 0.95 (std = 0.032)
    * R² (train) = 0.983
    * R² (test) = 0.918
Slightly overfitting as R² test < R² train considérant le std
* avec lasso 
    * alpha = 1 (par défaut) : R² = 0.96 (std = 0.024)
    * alpha = 0.1 : R² = 0.96 (std = 0.023)
    * R² (train) = 0.987
    * R² (test) = 0.923
* on a quand même le même nombre de variables (27), mais légèrement moins d'overfitting grâce à Lasso (Ridge n'a quasiement aucun effet)

In [308]:
result

,model,accuracy,set
0,LinearReg,0.987273,train
1,LinearReg,0.919191,test
2,Ridge,0.983418,train
3,Ridge,0.918564,test
4,Lasso,0.98675,train
5,Lasso,0.92336,test


In [309]:
out = result.sort_values(by=['set', 'accuracy'], ascending = [True, False])

In [310]:
px.bar(out, x = 'model', y= 'accuracy', color = 'set', barmode="group")

In [311]:
coef = coef.rename_axis('feature').reset_index()
coef['LinearReg_abs'] = abs(coef['LinearReg'])
coef['Ridge_abs'] = abs(coef['Ridge'])
coef['Lasso_abs'] = abs(coef['Lasso'])
coef_out = coef.melt(id_vars='feature', value_vars=['LinearReg_abs', 'Ridge_abs', 'Lasso_abs'],
        var_name='model', value_name='coef_value')
coef_out

,feature,model,coef_value
0,Temperature,LinearReg_abs,1546.522371
1,Fuel_Price,LinearReg_abs,49553.519801
2,CPI,LinearReg_abs,895505.770144
3,Unemployment,LinearReg_abs,34449.016493
4,day,LinearReg_abs,50224.750844
...,...,...,...
76,x0_17.0,Lasso_abs,271625.146450
77,x0_18.0,Lasso_abs,421909.327214
78,x0_19.0,Lasso_abs,783442.042906
79,x0_20.0,Lasso_abs,568537.887424


In [312]:
coef_out = coef_out.sort_values(by='coef_value', ascending = False)

In [313]:
px.bar(coef_out, x = 'feature', y= 'coef_value', color = 'model', barmode="group")